In [3]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

import re

from tqdm import tqdm

from datetime import datetime

import os 
import ast

In [4]:
data = pd.read_csv(r'/Users/andyhsu/Documents/GitHub/台灣代駕BMS/優惠券分析_分析/input/優惠券合併補齊資料.csv')
coupon = pd.read_csv(r'/Users/andyhsu/Documents/GitHub/台灣代駕BMS/優惠券分析_分析/input/coupon.csv')
register_time = pd.read_csv(r'/Users/andyhsu/Documents/GitHub/台灣代駕BMS/優惠券分析_分析/input/2020_2021_任務單會員註冊時間.csv')

In [5]:
data['Coupon_Title'] = data['Coupon_Title'].str.replace('折價','優惠')

In [6]:
coupon_list = [pd.DataFrame(coupon['Coupon_Title'].value_counts()).head(50).index][0]

In [7]:
coupon_list

Index(['註冊優惠', '評價獎勵', '優惠券', '邀請優惠', '50元優惠券', '信用卡綁定優惠', '嗨到集點-六點獎勵',
       '熟客驚喜回饋', '特約合作優惠', '嗨到集點-三點獎勵', '推播優惠', '15分未達優惠', '白銀會員升等禮', '邀請獎勵',
       '鑽石會員升等禮', '黃金會員升等禮', '台新卡友禮遇', '黑鑽會員升等禮', '白銀會員生日禮', 'line首次優惠',
       '幫朋友叫代駕獎勵', '嗨到集點-十點獎勵', '駕駛邀請優惠', '台新信用卡首用優惠', '再試一次優惠', 'VIP驚喜回饋',
       '200元優惠券', '黑鑽會員生日禮', '鑽石會員生日禮', '餐廳貼紙呼叫首用優惠', '黃金會員生日禮', '振興趟趟送優惠',
       '會員回饋', '會員計畫升級優惠', 'eTag合作優惠', '2021台新卡友禮遇', '兆豐頂級卡友2020優惠',
       '格上10公里優惠', '舊用戶完成搭乘每趟里程可折抵代駕金 NT100 元優惠禮遇', '幫朋友叫代駕首用獎勵', '熟客優惠券',
       '推播首用優惠200元', '花旗信用卡優惠', '新安東京海上獨享優惠', '兆豐卡友2020優惠', '我不依推播優惠',
       '格上租車合作優惠', '青商會員優待', '高雄首用200元', '白色野餐優惠'],
      dtype='object')

# 可以看到有部分的優惠券是取消單出現的

In [8]:
Cancel_Coupon = pd.DataFrame(data[(~pd.isna(data['Coupon_Title'])) & (data['取消統整'].str.contains('取消，'))]['Coupon_Title'].value_counts())

In [9]:
def 得到特定優惠券的使用到下一次使用其他優惠券交易(new_analyze_data,coupon_title,member_id):
    dataframe = new_analyze_data[new_analyze_data['member_id']==member_id]
    #只留成功單嗎還是取消也要看，如果只看成功要注意篩選完會有的dataframe是0？？
    dataframe = dataframe[dataframe['TaskState']==7]
    if len(dataframe) > 0:
        #print(len(dataframe))
        #先將Coupon_Title轉成string
        dataframe['createtime'] = pd.to_datetime(dataframe['createtime'],format = '%Y-%m-%d %H:%M:%S')
        dataframe.reset_index(drop=True,inplace=True)
        組合   = []
        dataframe['Coupon_Title'] = dataframe['Coupon_Title'].astype('str')
        最終結果 = pd.DataFrame({})
        # 先計算間隔時間
        #print(dataframe['Coupon_Title'].value_counts())
        if len(dataframe) > 1:
            for x in range(1,len(dataframe)):
                dataframe.loc[x,'間隔時間'] = dataframe.loc[x,'createtime'] - dataframe.loc[x-1,'createtime']
        else:
            dataframe['間隔時間'] = pd.Timestamp('NaT').to_pydatetime()



        使用特定優惠券的交易 = dataframe[dataframe['Coupon_Title'] == coupon_title].index
        #print(使用特定優惠券的交易)
        '''print(dataframe[dataframe['Coupon_Title'] == '嗨到集點-十點獎勵'].index)'''

        #這邊還是有使用優惠券，別搞混
        非使用特定優惠券的交易 = dataframe[(dataframe['Coupon_Title'] != 'nan') 
                                    & (dataframe['Coupon_Title'] != coupon_title) 
                                    & ~(np.isnat(dataframe['間隔時間']))].index

        #print(非使用特定優惠券的交易)

        # 為了解決非使用特定優惠券的交易的index都比使用特定優惠券的交易index小的問題，這邊先做一個處理
        if len(非使用特定優惠券的交易) > 0:
            if len(使用特定優惠券的交易)> 0 and 使用特定優惠券的交易[0] > 非使用特定優惠券的交易[-1]:
                非使用特定優惠券的交易 = []
            else:
                pass
        
        #print(使用特定優惠券的交易)
        #print(非使用特定優惠券的交易)
        #print(len(dataframe))
        if len(使用特定優惠券的交易) > 0:
            使用特定優惠券的交易 = list(使用特定優惠券的交易)
            非使用特定優惠券的交易 = list(非使用特定優惠券的交易)

            待刪除的資料 = []
            for x in range(len(使用特定優惠券的交易)-1):
                if 使用特定優惠券的交易[x] + 1 == 使用特定優惠券的交易[x+1]:
                    待刪除的資料.append(使用特定優惠券的交易[x])

            for y in 待刪除的資料:
                使用特定優惠券的交易.remove(y)

            #print(使用特定優惠券的交易) 
            #print(非使用特定優惠券的交易)
        
        
        ################------------------------------------以上必定沒問題

        # 代表沒用過該優惠券
        if len(使用特定優惠券的交易) == 0:
            pass
        
            return None

        elif len(使用特定優惠券的交易) != 0 and len(非使用特定優惠券的交易) != 0:
            if 使用特定優惠券的交易[-1] + 1 == len(dataframe):
                是否為最後一次使用 = 'Y'
            else:
                是否為最後一次使用 = 'N'
            
            for time_1 in 使用特定優惠券的交易:
                    for time_2 in 非使用特定優惠券的交易:
                        if time_1 + 1 == len(dataframe):
                            #print('最後一筆')
                            break
                        elif time_2 > time_1 : 
                            組合.append([time_1,time_2])
                            break
                        elif time_1 > 非使用特定優惠券的交易[-1]:
                            組合.append([time_1,len(dataframe)-1])
                            break


            #這邊還有待商榷
            組合_df        = pd.DataFrame(組合)

            #print(組合_df)

            #對組合_df做一些修正，不然他的交易排序跟我要的不一樣

            for x in range(len(組合_df)-1):
                if 組合_df.iloc[x+1,0] < 組合_df.iloc[x,1]:
                    組合_df.iloc[x,1] = 組合_df.iloc[x+1,0]
            #print(組合_df)
            交易組合_array                      =  組合_df.values
            使用該優惠券後製下次使用其他期間搭了幾次  = []
            單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern = pd.DataFrame()
            for 交易組合_single in list(交易組合_array):
                start      = 交易組合_single[0] + 1
                #因為我下面是用loc所以這邊減1才是正確的
                end        = 交易組合_single[1] - 1
                #print('start:',start)
                #print('end:',end)
                if start <= end:
                    #因為我不看他首次使用優惠券的間隔時間
                    想看的交易單_間隔時間 = dataframe.loc[start:end,:]
                    #不看最後一次，因為最後一次是他去使用其他優惠券
                    使用該優惠券後製下次使用其他期間搭了幾次.append(str(len(想看的交易單_間隔時間)))
                    單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern = 單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern\
                                                                                .append(想看的交易單_間隔時間)
            
                    #print(單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern)
                    #print(使用該優惠券後製下次使用其他期間搭了幾次)
                # classic example id: 99132 邀請優惠
                else:
                    #因為我不看他首次使用優惠券的間隔時間
                    想看的交易單_間隔時間 = dataframe.loc[start:end,:]
                    #不看最後一次，因為最後一次是他去使用其他優惠券
                    使用該優惠券後製下次使用其他期間搭了幾次.append(str(0))
                    #分支_result = pd.DataFrame([[member_id,'中間沒有間隔',dataframe['間隔時間'].mean()\
                        #,使用該優惠券後製下次使用其他期間搭了幾次,是否為最後一次使用]])
                    單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern = 單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern.append(None)

                if len(單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern) != 0 :
                    分支_result = pd.DataFrame([[member_id,單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern['間隔時間'].mean(),dataframe['間隔時間'].mean()\
                            ,使用該優惠券後製下次使用其他期間搭了幾次,是否為最後一次使用]])
                else:
                    分支_result = pd.DataFrame([[member_id,'中間沒有間隔',dataframe['間隔時間'].mean(),使用該優惠券後製下次使用其他期間搭了幾次,是否為最後一次使用]])
            #print(分支_result)
                

        ################------------------------------------以上必定沒問題
        elif len(使用特定優惠券的交易) != 0 and len(非使用特定優惠券的交易) == 0:
            if 使用特定優惠券的交易[-1] + 1 == len(dataframe):
                是否為最後一次使用 = 'Y'
                #print('最後一筆')
            else:
                是否為最後一次使用 = 'N'

            組合 = []
            使用該優惠券後製下次使用其他期間搭了幾次  = []
            #classic example id: 98621 白銀會員升等禮, 39294 特約合作優惠
            if len(使用特定優惠券的交易) == 1 and 使用特定優惠券的交易[0]+1 == len(dataframe):
                #print('最後一筆')
                分支_result = pd.DataFrame([[member_id,'之後沒有呼叫',dataframe['間隔時間'].mean()\
                ,[0],是否為最後一次使用]])
                #print(分支_result)

            #classic example id: 163845 註冊優惠
            elif len(使用特定優惠券的交易) == 1:
                #想看的交易單_間隔時間 = dataframe.loc[使用特定優惠券的交易[0]:,:]
                組合.append([使用特定優惠券的交易[0],len(dataframe)])
                單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern = dataframe.loc[組合[0][0]+1:組合[0][1],:]
                #print(單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern)
                分支_result = pd.DataFrame([[member_id,單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern['間隔時間'].mean(),dataframe['間隔時間'].mean()\
                ,[len(單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern)],是否為最後一次使用]])
                #print(分支_result)
                

            elif len(使用特定優惠券的交易) >= 2:
                單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern = pd.DataFrame({})
                if len(使用特定優惠券的交易) == 2:
                    for x in [0]:
                        組合.append([使用特定優惠券的交易[x],使用特定優惠券的交易[x+1]])
                    #這是要方便計算最後一次使用該優惠券後的消費記錄
                    組合.append([使用特定優惠券的交易[-1],len(dataframe)])
                else:
                    for x in range(len(使用特定優惠券的交易)-2):
                        #這是代表使用特定優惠券的那些訂單
                        組合.append([使用特定優惠券的交易[x],使用特定優惠券的交易[x+1]])
                    #這是要方便計算最後一次使用該優惠券後的消費記錄
                    組合.append([使用特定優惠券的交易[-1],len(dataframe)])
                #print(組合)
                for x in 組合:
                    使用該優惠券後製下次使用其他期間搭了幾次.append(str(x[1]-x[0]-1))

                    if x[1] == x[0] + 1:
                        #單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern = 單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern.append({})
                        pass
                    else:  
                        單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern = 單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern.\
                            append(dataframe.loc[x[0]+1:x[1]-1,:])
                #print(使用該優惠券後製下次使用其他期間搭了幾次)

                if len(單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern) == 0:
                    #print('here')
                    分支_result = pd.DataFrame([[member_id,'中間沒有間隔',dataframe['間隔時間'].mean()\
                ,使用該優惠券後製下次使用其他期間搭了幾次,是否為最後一次使用]])

                else:
                    分支_result = pd.DataFrame([[member_id,單一會員使用特定優惠券後製使用其他優惠券_簡格使用時間pattern['間隔時間'].mean(),dataframe['間隔時間'].mean()\
                ,使用該優惠券後製下次使用其他期間搭了幾次,是否為最後一次使用]])
        return 分支_result

    else:
        pass
                
得到特定優惠券的使用到下一次使用其他優惠券交易(new_analyze_data=data,coupon_title = '200元優惠券',member_id = 117683)

,0,1,2,3,4
0,117683,18 days 10:01:18,26 days 09:46:08.181818182,"[2, 4, 4]",N


# 有些有優惠券但沒有優惠金額，將其補上

In [10]:
折價金額_dict = {
    '15分未達優惠':100,
    '50元優惠券':50,
    '200元優惠券':200,
    '白色野餐優惠':200,
    '再試一次優惠':'再試一次',
    '我不依推播優惠':100,
    '青商會員優待':50,
    '振興趟趟送優惠':50,
    '推播首用優惠200元':200,
    '推播優惠':100,
    '黃金會員生日禮':100,
    '黑鑽會員生日禮':100,
    '鑽石會員生日禮':100,
    '會員回饋':100,
    '熟客優惠券':100,
    '熟客驚喜回饋':350,
    '優惠券':'優惠券',
    '舊用戶完成搭乘每趟里程可折抵代駕金 NT100 元優惠禮遇':100,
    'VIP驚喜回饋':350
}

In [11]:
data[(~pd.isna(data['Coupon_Title'])) & ((data['UserCreditValue']==0) | (np.isnan(data['UserCreditValue']==0)))][['Coupon_Title','UserCreditValue']]

,Coupon_Title,UserCreditValue
29997,15分未達優惠,0.0
31753,15分未達優惠,0.0
41973,15分未達優惠,0.0
43717,15分未達優惠,0.0
145468,50元優惠券,0.0


In [12]:
data.loc[29997,'UserCreditValue'] = 100
data.loc[31753,'UserCreditValue'] = 100
data.loc[43717,'UserCreditValue'] = 100
data.loc[41973,'UserCreditValue'] = 100
data.loc[145468,'UserCreditValue'] = 50
# Double Check
data[(~pd.isna(data['Coupon_Title'])) & ((data['UserCreditValue']==0) | (np.isnan(data['UserCreditValue']==0)))][['Coupon_Title','UserCreditValue']]

,Coupon_Title,UserCreditValue


In [13]:
for x in data['Coupon_Title'].unique():
    if pd.isna(x) != True:
        if '註冊' in x:
            print(x)
    else:
        pass

註冊優惠


In [14]:
register_time.drop_duplicates(subset=['Member_ID'],inplace=True)

# 註冊優惠

In [15]:
data_註冊優惠 = data[(data['Coupon_Title']=='註冊優惠') & (data['取消統整']=='未取消')]
print(len(data_註冊優惠))
data_註冊優惠_ = pd.merge(data_註冊優惠,register_time,left_on='member_id',right_on='Member_ID',how='left')
print(len(data_註冊優惠_))
data_註冊優惠_

13674
13674


,id,member_id,driver_id,TaskState,type,call_type,TaskRideTS,createtime,UserLat,UserLon,...,TaskDistanceFee,is_pay,pay_type,is_first_use,Color,取消統整,Task_ID,Coupon_Title,Member_ID,Member_Createtime
0,305968,99706,1049,7,1,1,1577809105,2020-01-01 00:19:00,24.146,120.663,...,0.0,1,1,1,#8fb47e,未取消,305968.0,註冊優惠,99706,2020-01-01 00:16:50
1,305981,38787,1584,7,1,1,1577809588,2020-01-01 00:26:00,25.000,121.504,...,0.0,1,1,0,#8fb47e,未取消,305981.0,註冊優惠,38787,2018-05-25 00:14:34
2,306005,99711,2412,7,1,1,1577810006,2020-01-01 00:33:00,25.050,121.576,...,450.0,1,1,1,#8fb47e,未取消,306005.0,註冊優惠,99711,2020-01-01 00:25:47
3,306038,99717,1946,7,1,1,1577810739,2020-01-01 00:45:00,25.013,121.297,...,0.0,1,1,1,#8fb47e,未取消,306038.0,註冊優惠,99717,2020-01-01 00:38:19
4,306050,99722,2409,7,1,1,1577810908,2020-01-01 00:48:00,25.080,121.558,...,0.0,1,2,1,#8fb47e,未取消,306050.0,註冊優惠,99722,2020-01-01 00:45:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13669,716743,169547,2654,7,1,1,1624905192,2021-06-29 02:33:00,24.969,121.518,...,1150.0,1,1,0,#8fb47e,未取消,716743.0,註冊優惠,169547,2021-05-19 02:50:41
13670,716817,170287,2270,7,1,1,1624953956,2021-06-29 16:06:00,25.056,121.591,...,0.0,1,1,1,#8fb47e,未取消,716817.0,註冊優惠,170287,2021-06-29 16:05:21
13671,716825,170288,2309,7,1,1,1624966384,2021-06-29 19:33:00,25.065,121.527,...,100.0,1,1,1,#8fb47e,未取消,716825.0,註冊優惠,170288,2021-06-29 19:30:25
13672,716879,170299,1803,7,1,1,1624980844,2021-06-29 23:34:00,24.981,121.485,...,1250.0,1,2,1,#8fb47e,未取消,716879.0,註冊優惠,170299,2021-06-29 23:29:16


In [16]:
member_df = pd.DataFrame(data_註冊優惠['member_id'].value_counts())
member_df[member_df['member_id']>=14]

,member_id


In [17]:
註冊優惠_pattern = pd.DataFrame()
for x in data_註冊優惠_.member_id.unique():
    a = 得到特定優惠券的使用到下一次使用其他優惠券交易(new_analyze_data=data,coupon_title = '註冊優惠',member_id = x)
    註冊優惠_pattern = 註冊優惠_pattern.append(a)

In [18]:
註冊優惠_pattern = 註冊優惠_pattern.reset_index(drop=True)
註冊優惠_pattern.columns = ['member_id','平均使用服務間隔（使用優惠券後）','平均使用服務間隔（整體）','使用間隔','是否為最後一次使用']
註冊優惠_pattern['成為忠實用戶'] = 註冊優惠_pattern['使用間隔'].apply(lambda x: True if int(x[0]) >= 2 else False)
註冊優惠_pattern

,member_id,平均使用服務間隔（使用優惠券後）,平均使用服務間隔（整體）,使用間隔,是否為最後一次使用,成為忠實用戶
0,99706,之後沒有呼叫,NaT,[0],Y,False
1,38787,166 days 00:35:00,166 days 00:35:00,[1],N,False
2,99711,之後沒有呼叫,NaT,[0],Y,False
3,99717,419 days 21:18:00,419 days 21:18:00,[1],N,False
4,99722,159 days 18:18:45,159 days 18:18:45,[4],N,True
...,...,...,...,...,...,...
13651,169547,21 days 07:40:20,28 days 04:27:24,[3],N,True
13652,170287,之後沒有呼叫,NaT,[0],Y,False
13653,170288,之後沒有呼叫,NaT,[0],Y,False
13654,170299,之後沒有呼叫,NaT,[0],Y,False


In [19]:
註冊優惠_pattern[註冊優惠_pattern['成為忠實用戶']==True]

,member_id,平均使用服務間隔（使用優惠券後）,平均使用服務間隔（整體）,使用間隔,是否為最後一次使用,成為忠實用戶
4,99722,159 days 18:18:45,159 days 18:18:45,[4],N,True
5,99728,80 days 16:00:00,80 days 16:00:00,[3],N,True
6,92231,22 days 18:38:45,22 days 18:38:45,[32],N,True
8,93729,3 days 15:13:20,90 days 00:06:37.500000,[3],N,True
9,99248,9 days 12:19:30,13 days 18:02:38.076923077,[4],N,True
...,...,...,...,...,...,...
13639,170183,53 days 14:24:20,53 days 14:24:20,[3],N,True
13642,170199,62 days 16:29:00,62 days 16:29:00,[3],N,True
13643,170244,15 days 22:40:30,15 days 22:40:30,[2],N,True
13644,170251,23 days 03:55:30,23 days 03:55:30,[6],N,True


##### 轉成時間型態

In [20]:
data_註冊優惠_['createtime'] = pd.to_datetime(data_註冊優惠_['createtime'])
data_註冊優惠_['Member_Createtime'] = pd.to_datetime(data_註冊優惠_['Member_Createtime'])

In [21]:
data_註冊優惠_['註冊優惠_間隔'] = data_註冊優惠_['createtime'] - data_註冊優惠_['Member_Createtime']

In [22]:
data_註冊優惠_去重複 = data_註冊優惠_.drop_duplicates(subset='member_id',keep='first')

### 將註冊優惠及註冊優惠_間隔合併

In [23]:
註冊優惠_pattern_忠實客戶_註冊優惠_間隔 = 註冊優惠_pattern.merge(data_註冊優惠_去重複[['member_id','註冊優惠_間隔']],on='member_id')

In [24]:
註冊優惠_pattern_忠實客戶_註冊優惠_間隔['註冊優惠_間隔'].mean()

Timedelta('30 days 17:30:07.027606914')

In [25]:
註冊優惠_pattern_忠實客戶_註冊優惠_間隔['註冊優惠_間隔_天'] = 註冊優惠_pattern_忠實客戶_註冊優惠_間隔['註冊優惠_間隔'].apply(lambda x:x.days)

In [26]:
註冊優惠_pattern_忠實客戶_註冊優惠_間隔[註冊優惠_pattern_忠實客戶_註冊優惠_間隔['註冊優惠_間隔_天']==-1]

,member_id,平均使用服務間隔（使用優惠券後）,平均使用服務間隔（整體）,使用間隔,是否為最後一次使用,成為忠實用戶,註冊優惠_間隔,註冊優惠_間隔_天
5,99728,80 days 16:00:00,80 days 16:00:00,[3],N,True,-1 days +23:59:43,-1
30,99810,之後沒有呼叫,NaT,[0],Y,False,-1 days +23:59:57,-1
58,99858,之後沒有呼叫,NaT,[0],Y,False,-1 days +23:59:50,-1
65,99876,之後沒有呼叫,NaT,[0],Y,False,-1 days +23:59:56,-1
241,100895,327 days 23:28:00,327 days 23:28:00,[1],N,False,-1 days +23:59:56,-1
...,...,...,...,...,...,...,...,...
13521,169298,92 days 01:49:00,92 days 01:49:00,[2],N,True,-1 days +23:59:33,-1
13584,169638,之後沒有呼叫,NaT,[0],Y,False,-1 days +23:59:59,-1
13587,169651,7 days 23:50:00,7 days 23:50:00,[1],N,False,-1 days +23:59:48,-1
13607,169831,之後沒有呼叫,NaT,[0],Y,False,-1 days +23:59:38,-1


In [27]:
註冊優惠_pattern_忠實客戶_註冊優惠_間隔.groupby('成為忠實用戶').agg({'註冊優惠_間隔_天':'mean'})

,註冊優惠_間隔_天
成為忠實用戶,
False,24.449790
True,47.370258


In [28]:
a = 註冊優惠_pattern_忠實客戶_註冊優惠_間隔[['註冊優惠_間隔_天','成為忠實用戶','member_id']].groupby(['註冊優惠_間隔_天','成為忠實用戶']).count()

In [43]:
b = a.reset_index()[4:80]

In [44]:
b

,註冊優惠_間隔_天,成為忠實用戶,member_id
4,1,False,170
5,1,True,52
6,2,False,95
7,2,True,40
8,3,False,65
...,...,...,...
75,36,True,7
76,37,False,6
77,37,True,6
78,38,True,6


In [31]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
from plotly.subplots import make_subplots

In [41]:
c = a[4:].reset_index()
d = c[c['成為忠實用戶']==True][:101]
e = c[c['成為忠實用戶']==False][:101]

In [39]:
d

,註冊優惠_間隔_天,成為忠實用戶,member_id
1,1,True,52
3,2,True,40
5,3,True,42
7,4,True,35
9,5,True,36
...,...,...,...
197,110,True,2
199,111,True,1
201,112,True,1
203,113,True,2


In [45]:
fig = make_subplots(rows=3, cols=1,row_heights = [0.4,0.3,0.3],subplot_titles = ('使用註冊券的時間跟成為忠實客戶的關係','成為忠實用戶 縱軸(day) 橫軸(人)','未成為忠實用戶 縱軸(day) 橫軸(人)'),
                    specs = [[{"secondary_y": False}],
                             [{"secondary_y": False}],
                             [{"secondary_y": False}]],
                    shared_xaxes=False)

fig.add_trace(go.Bar(x=b[b['成為忠實用戶']==True]['註冊優惠_間隔_天'].astype('str'),y=b[b['成為忠實用戶']==True].member_id,name='成為忠實用戶',marker_color='#20364a'),row=1,col=1)

fig.add_trace(go.Bar(x=b[b['成為忠實用戶']==False]['註冊優惠_間隔_天'].astype('str'),y=b[b['成為忠實用戶']==False].member_id,name='未成為忠實用戶',marker_color='#3f87c8'),row=1,col=1)

fig.add_trace(go.Scatter(x=d['註冊優惠_間隔_天'].astype('str'),y=d.member_id,name='成為忠實用戶',marker_color='#20364a'),row=2,col=1)
fig.add_trace(go.Scatter(x=e['註冊優惠_間隔_天'].astype('str'),y=e.member_id,name='未成為忠實用戶',marker_color='#3f87c8'),row=3,col=1)

fig.update_layout(title=dict(
    font = dict(
        color = 'blue',
        size  = 23
    ),
    text = '註冊券',
),
height=650,width=1000)

fig.for_each_xaxis(lambda x: x.update(showgrid=False))
fig.for_each_yaxis(lambda x: x.update(showgrid=False))
fig.show()

# 優惠券總整理 包含 優惠券的使用情形及成效

In [354]:
def 優惠券pattern_總整理(x):
    data_特定優惠 = data[(data['Coupon_Title']== x) & (data['取消統整']=='未取消')]
    data_特定優惠_ = pd.merge(data_特定優惠,register_time,left_on='member_id',right_on='Member_ID',how='left')

    data_特定優惠_pattern = pd.DataFrame()
    for y in data_特定優惠_.member_id.unique():
        a = 得到特定優惠券的使用到下一次使用其他優惠券交易(new_analyze_data=data,coupon_title = x,member_id = y)
        data_特定優惠_pattern = data_特定優惠_pattern.append(a)
    data_特定優惠_pattern.to_csv('/Users/andyhsu/Documents/GitHub/台灣代駕BMS/優惠券分析_分析/output/'+x+'_資料.csv',index=False,header=True,encoding='utf-8-sig')


for z in coupon_list:
    優惠券pattern_總整理(z)

In [ ]:
warnings.filterwarnings('ignore')
output_folder = os.path.join(os.getcwd(),'output')
for file in os.listdir(output_folder):
    if '_資料.csv' in file:
        temp = pd.read_csv(output_folder + '/' +file,encoding='utf-8-sig')
        #print(temp)
        temp.columns = ['member_id','平均使用服務間隔（使用優惠券後）','平均使用服務間隔（整體）','使用間隔','是否為最後一次使用']
        def compare_time(time_1,time_2):
            if time_1  == '中間沒有間隔' or time_1 == '之後沒有呼叫':
                return '無法比較'
            else:
                def get_date(x):
                    if re.match(r'(\d+)(\sdays)',x):
                            days = int(re.match(r'(\d+)(\sdays)',x).group(1))
                            hours = int(re.search(r'(\d+):(\d+):(\d+)',x).group(1))
                            mins = int(re.search(r'(\d+):(\d+):(\d+)',x).group(2))
                            seconds = int(re.search(r'(\d+):(\d+):(\d+)',x).group(3))
                            d = datetime.timedelta(days=days,hours=hours,minutes=mins,seconds=seconds)
                    return datetime.datetime.today() + d

                time_1_d = get_date(time_1)
                time_2_d = get_date(time_2)

                if time_1_d < time_2_d:
                    return True
                else:
                    return False

        temp['降低使用時間'] = temp.apply(lambda x: compare_time(x['平均使用服務間隔（使用優惠券後）'],x['平均使用服務間隔（整體）']),axis=1)
        temp.to_csv(output_folder + '/' +file,encoding='utf-8-sig',index=False,header=True)


會員降低時間整理=pd.DataFrame()
for file in os.listdir(output_folder):
    if '_資料.csv' in file:
        temp = pd.read_csv(output_folder + '/' +file,encoding='utf-8-sig')
        temp_data = temp[['member_id','降低使用時間']]
        temp_data['Coupon'] = file.replace('_資料.csv','')
        會員降低時間整理 = 會員降低時間整理.append(temp_data)


#整理會員使用優惠券的情況
會員使用優惠券整理 = []
for x,y in 會員降低時間整理.groupby('member_id'):
    會員使用優惠券整理.append([x, [x for x in y['Coupon'].values]])

#整理會員使用優惠券有效的情況
會員使用優惠券有效_整理 = []
會員降低時間整理_   =  會員降低時間整理[會員降低時間整理['降低使用時間']=='True']
for x,y in 會員降低時間整理_.groupby('member_id'):
    會員使用優惠券有效_整理.append([x, [x for x in y['Coupon'].values]])

c = pd.DataFrame(會員使用優惠券整理,columns=['member_id','使用的優惠券']).merge(pd.DataFrame(會員使用優惠券有效_整理,columns=['member_id','使用的有效優惠券']),on='member_id',how='outer')


d = 會員降低時間整理.groupby(['member_id','降低使用時間']).agg({'降低使用時間':'count'}).unstack()

d.merge(c,on='member_id',how='left').to_csv(r'/Users/andyhsu/Documents/GitHub/台灣代駕BMS/優惠券分析_分析/output/優惠券綜合整理.csv',index=False,header=True,encoding='utf-8-sig')

In [ ]:
import datetime
output_folder = os.path.join(os.getcwd(),'output')
final_result_lost          = []
final_result_induce        = []
final_result_decrease_time = []
for file in os.listdir(output_folder):
    if '_資料.csv' in file:
        temp = pd.read_csv(output_folder + '/' +file,encoding='utf-8-sig')
        temp = temp.reset_index(drop=True)
        temp.columns = ['member_id','平均使用服務間隔（使用優惠券後）','平均使用服務間隔（整體）','使用間隔','是否為最後一次使用','降低使用時間']
        def 忠實用戶_identify(x):
            x = ast.literal_eval(x)
            for y in x:
                if int(y)>=2:
                    return True
                    break
                else:
                    pass
            return False
        temp['成為忠實用戶'] = temp['使用間隔'].apply(lambda x: 忠實用戶_identify(x))
        Y    = len(temp[temp['是否為最後一次使用'] == 'Y'])
        N    = len(temp[temp['是否為最後一次使用'] == 'N'])
        temp_list = [file.replace('_資料.csv',''),Y,N]
        final_result_lost.append(temp_list)
        ###-------------------------------###
        a = []
        b = 0
        for x,y in list(zip(temp['member_id'],temp['使用間隔'])):
            z = ast.literal_eval(y)
            for n in z:
                if int(n) >= 5:
                    b += 1
                    a.append(x)
        final_result_induce.append([file.replace('_資料.csv',''),list(set(a)),b,len(list(set(a))),len(temp)])

        ###-------------------------------###
        def compare_time(time_1,time_2):
            if time_1  == '中間沒有間隔' or time_1 == '之後沒有呼叫':
                return '無法比較'
            else:
                def get_date(x):
                    if re.match(r'(\d+)(\sdays)',x):
                        days = int(re.match(r'(\d+)(\sdays)',x).group(1))
                        hours = int(re.search(r'(\d+):(\d+):(\d+)',x).group(1))
                        mins = int(re.search(r'(\d+):(\d+):(\d+)',x).group(2))
                        seconds = int(re.search(r'(\d+):(\d+):(\d+)',x).group(3))
                        d = datetime.timedelta(days=days,hours=hours,minutes=mins,seconds=seconds)
                    return datetime.datetime.today() + d

                time_1_d = get_date(time_1)
                time_2_d = get_date(time_2)

                if time_1_d < time_2_d:
                    return True
                else:
                    return False
                    
        temp['是否有降低時間'] = temp.apply(lambda x:compare_time(x['平均使用服務間隔（使用優惠券後）'],x['平均使用服務間隔（整體）']),axis=1)
        T = len(temp[temp['是否有降低時間'] == True])
        F = len(temp[temp['是否有降低時間'] == False])
        無法比較 = len(temp[temp['是否有降低時間'] == '無法比較'])
        final_result_decrease_time.append([file.replace('_資料.csv',''),T,F,無法比較])

        ###-------------------------------###
    #將以上的list轉成dataframe並合併
    coupon_df = pd.DataFrame(final_result_lost,columns = ['優惠券','最後使用_Y','最後使用_N'])
    coupon_df['流失率'] = coupon_df['最後使用_Y'] / (coupon_df['最後使用_Y'] + coupon_df['最後使用_N'])
    final_result_df = pd.DataFrame(\
        final_result_induce,columns = ['優惠券','觸發的人（非重複）','觸發次數','觸發人數（非重複）','總使用人數'])
    final_result_decrease_time_df = pd.DataFrame(\
        final_result_decrease_time,columns = ['優惠券','降低使用間隔的人數','提升使用間隔的人數','無法比較'])

優惠券相關_df = coupon_df.merge(final_result_df,on='優惠券')
優惠券相關_df2 = 優惠券相關_df.merge(final_result_decrease_time_df,on='優惠券')

In [370]:
優惠券相關_df2.to_csv(r'/Users/andyhsu/Documents/GitHub/台灣代駕BMS/優惠券分析_分析/output/優惠券使用情形分析.csv',index=False,header=True,encoding='utf-8-sig')

In [48]:
優惠券相關_df2 = pd.read_csv('/Users/andyhsu/Documents/GitHub/台灣代駕BMS/優惠券分析_分析/output/優惠券使用情形分析.csv')

In [49]:
優惠券相關_df2.sort_values(by=['流失率'])

,優惠券,最後使用_Y,最後使用_N,流失率,觸發的人（非重複）,觸發次數,觸發人數（非重複）,總使用人數,降低使用間隔的人數,提升使用間隔的人數,無法比較
30,青商會員優待,0,21,0.000000,"[25632, 6723, 7620, 91184, 101424, 67321, 8383...",8,8,21,11,6,4
19,黑鑽會員生日禮,0,87,0.000000,"[10368, 120332, 14097, 60573, 4520, 53416, 107...",22,22,87,15,29,43
10,熟客優惠券,0,64,0.000000,"[10368, 35343, 59414, 66207, 14496, 32168, 540...",23,23,64,34,11,19
22,黑鑽會員升等禮,1,306,0.003257,"[13440, 39809, 1026, 87682, 51332, 97796, 1191...",56,56,307,52,77,178
43,鑽石會員生日禮,1,151,0.006579,"[29441, 101635, 18696, 91658, 24077, 51342, 52...",32,32,152,28,46,78
46,嗨到集點-十點獎勵,2,299,0.006645,"[97921, 77442, 37762, 32007, 27273, 73867, 472...",68,65,301,93,46,162
4,會員計畫升級優惠,1,147,0.006757,"[21760, 37408, 2920, 45465, 90476, 8461, 12397...",14,14,148,44,32,72
47,鑽石會員升等禮,8,675,0.011713,"[43523, 35844, 65541, 42503, 65038, 25620, 706...",149,149,683,151,194,338
15,振興趟趟送優惠,2,167,0.011834,"[98048, 55937, 66178, 101635, 68484, 63752, 76...",58,58,169,75,41,53
0,嗨到集點-六點獎勵,12,908,0.013043,"[23552, 4608, 35844, 65541, 16391, 42503, 9225...",203,186,920,199,230,491


In [50]:
coupon_發放 = pd.DataFrame(coupon.Coupon_Title.value_counts())

In [51]:
coupon_使用 = pd.DataFrame(data[data['TaskState']==7].Coupon_Title.value_counts())

In [52]:
coupon_發放使用整理 = coupon_發放.merge(coupon_使用,right_index=True,left_index=True)
coupon_發放使用整理.columns=['發放','使用']
coupon_發放使用整理['use_rate'] = coupon_發放使用整理['使用'] / coupon_發放使用整理['發放']

In [62]:
data_test = 優惠券相關_df2[優惠券相關_df2['優惠券'].isin(['白銀會員生日禮','黃金會員生日禮','鑽石會員生日禮','黑鑽會員生日禮'])]
data_test = data_test.set_index('優惠券').loc[['白銀會員生日禮','黃金會員生日禮','鑽石會員生日禮','黑鑽會員生日禮'],:].reset_index()
coupon_發放使用整理_test = coupon_發放使用整理[coupon_發放使用整理.index.isin(['白銀會員生日禮','黃金會員生日禮','鑽石會員生日禮','黑鑽會員生日禮'])]
coupon_發放使用整理_test = coupon_發放使用整理_test.loc[['白銀會員生日禮','黃金會員生日禮','鑽石會員生日禮','黑鑽會員生日禮'],:]

In [64]:
fig = make_subplots(rows=5, cols=1,row_heights = [0.2,0.2,0.25,0.2,0.15],subplot_titles = ('優惠券流失率','是否為會員最後使用之服務','降低服務使用間隔的成效','觸發人數','發放及領用情形'),
                    specs = [[{"secondary_y": False}],
                             [{"secondary_y": False}],
                             [{"secondary_y": False}],
                             [{"secondary_y": False}],
                             [{"secondary_y": True}]],
                    shared_xaxes=True)



fig.add_trace(go.Scatter(x=data_test.優惠券,y=data_test.流失率,name='流失率',marker_color='#3f87c8'),row=1,col=1)
fig.add_trace(go.Scatter(x=data_test.優惠券,y=data_test['觸發人數（非重複）'],name='觸發人數（非重複）',marker_color='#3f87c8'),row=4,col=1)


fig.add_trace(go.Bar(x=data_test.優惠券,y=data_test['最後使用_Y'],name='最後使用人數',marker_color='#20364a',offsetgroup=1,text=data_test['最後使用_Y']),row=2,col=1)
fig.add_trace(go.Bar(x=data_test.優惠券,y=data_test['最後使用_N'],name='非最後使用人數',marker_color='#3f87c8',offsetgroup=0),row=2,col=1)


fig.add_trace(go.Scatter(x=data_test.優惠券,y=data_test.降低使用間隔的人數,name='降低使用間隔的人數',marker_color='#3f87c8'),row=3,col=1)
fig.add_trace(go.Scatter(x=data_test.優惠券,y=data_test.提升使用間隔的人數,name='提升使用間隔的人數',marker_color='#591414'),row=3,col=1)
fig.add_trace(go.Scatter(x=data_test.優惠券,y=data_test.無法比較,name='無法比較的人數',marker_color='#B3C379'),row=3,col=1)

fig.add_trace(go.Bar(x=coupon_發放使用整理_test.index,y=coupon_發放使用整理_test['發放'],name='發放',marker_color='#20364a',offsetgroup=1,text=data_test['最後使用_Y']),row=5,col=1)
fig.add_trace(go.Bar(x=coupon_發放使用整理_test.index,y=coupon_發放使用整理_test['使用'],name='使用',marker_color='#3f87c8',offsetgroup=0),row=5,col=1)
fig.add_trace(go.Scatter(x=coupon_發放使用整理_test.index,y=coupon_發放使用整理_test['use_rate'],name='使用率',marker_color='#d09b63'),row=5,col=1,secondary_y=True)



fig.update_layout(title=dict(
    font = dict(
        color = 'blue',
        size  = 23
    ),
    text = '優惠券視覺化',
),
height=720,width=1200)

fig.for_each_xaxis(lambda x: x.update(showgrid=False))
fig.for_each_yaxis(lambda x: x.update(showgrid=False))
config = dict({'scrollZoom': True})
fig.show(config=config)